In [ ]:
from collections import deque
def Brandes_CB_algoritm(G):

    C_B = {v: 0 for v in G.nodes()} #мера связности для каждого узла
    V = G.nodes()

    for s in V:
      S = []
      P = {w: [] for w in V}

      sigma = {t: 0 for t in V} #кол-во кратчайших путей
      sigma[s] = 1

      d = {t: -1 for t in V} #минимальное расстояние
      d[s] = 0

      Q = deque()
      Q.append(s)

      while Q:
        v = Q.popleft() #извлекает из очереди вершину v
        S.append(v) #кладет переменую v
        for w in G.neighbors(v):
          #w нашлась первый раз?
          if d[w] < 0:
            Q.append(w)
            d[w] = d[v] + 1
          #кратчайший путь к w через v?
          if d[w] == d[v] + 1:
            sigma[w] = sigma[w] + sigma[v]
            P[w].append(v)


      delta = {v: 0 for v in V}
      # S возвращает вершины в порядке возрастания растаяния от s
      while S:
        w = S.pop()
        for v in P[w]:
          delta[v] = delta[v] + (sigma[v] / sigma[w]) * (1 + delta[w])
        if w != s:
          C_B[w] = (C_B[w] + delta[w]/2)

    print(C_B)


In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
from ipywidgets import interact
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.colors as mcolors
import matplotlib.colors as colors
import numpy as np
import matplotlib.cm as cm

def Interface_BC1(G, algorithm,k1,size_i):
    # Create widget outputs
    graph_output = widgets.Output()
    centrality_output = widgets.Output()

    def draw_graph():
        with graph_output:
            clear_output(wait=True)
            pos = nx.spring_layout(G, k=k1, iterations=50, seed=42)
            plt.figure(figsize=(size_i, size_i))

            # Calculate betweenness centrality
            betweenness = nx.betweenness_centrality(G, normalized=False)


            # Determine color divisions
            if highlight_checkbox.value:
                max_betweenness = max(betweenness.values())
                min_betweenness = min(betweenness.values())

                if max_betweenness != min_betweenness:
                    step = (max_betweenness - min_betweenness) / 9  # Divide into 10 intervals
                else:
                    step = 0

                # Define a colormap from light to dark red
                colormap = cm.Reds  # Using Reds colormap for red shades
                norm = colors.Normalize(vmin=min_betweenness, vmax=max_betweenness)

                node_colors = []
                for node in G.nodes():
                    centrality_value = betweenness.get(node, 0)
                    if step > 0:
                        color_index = int((centrality_value - min_betweenness) / step)
                        color = colormap(norm(centrality_value))
                    else:
                        color = colormap(norm(centrality_value))
                    node_colors.append(color)
            else:
                node_colors = 'lightblue'

            nx.draw(G, pos, with_labels=True, node_color=node_colors, node_size=200, font_size=10, font_color='black', edge_color='gray')

            # Display betweenness centrality values near nodes
            labels = {node: f"\n\n\n\n{value:.2f}" for node, value in betweenness.items()}
            nx.draw_networkx_labels(G, pos, labels, font_size=8, font_color='black')
            plt.show()

    def add_node(node):
        G.add_node(int(node))
        draw_graph()
        node_input.value = ""

    def add_edge(node1, node2):
        G.add_edge(int(node1), int(node2))
        draw_graph()
        edge_input.value = ""

    def remove_node(node):
        if node in G.nodes:
            G.remove_node(int(node))
        draw_graph()
        node_input.value = ""

    def remove_edge(node1, node2):
        if G.has_edge(node1, node2):
            G.remove_edge(int(node1), int(node2))
        draw_graph()
        node_input.value = ""

    def highlight_checkbox_changed(change):
        draw_graph()

    # Checkbox for highlighting nodes by centrality
    highlight_checkbox = widgets.Checkbox(value=False, description="Highlight by Betweenness Centrality", style={'description_width': 'initial'})
    highlight_checkbox.observe(highlight_checkbox_changed, names='value')

    # Input widgets for node and edge addition/removal
    node_input = widgets.Text(description="Node")
    edge_input = widgets.Text(description="Edge", layout=widgets.Layout(margin='30px 0 0 0'))

    # Buttons for adding and removing nodes and edges
    add_node_button = widgets.Button(description="Add Node")
    add_edge_button = widgets.Button(description="Add Edge")

    remove_node_button = widgets.Button(description="Remove Node")
    remove_edge_button = widgets.Button(description="Remove Edge")

    # Event handlers for buttons
    def add_node_button_clicked(b):
        add_node(node_input.value)

    def add_edge_button_clicked(b):
        nodes = edge_input.value.split()
        if len(nodes) == 2:
            add_edge(nodes[0], nodes[1])

    def remove_node_button_clicked(b):
        remove_node(node_input.value)

    def remove_edge_button_clicked(b):
        nodes = edge_input.value.split()
        if len(nodes) == 2:
            remove_edge(nodes[0], nodes[1])

    # Assign event handlers to buttons
    add_node_button.on_click(add_node_button_clicked)
    add_edge_button.on_click(add_edge_button_clicked)
    remove_node_button.on_click(remove_node_button_clicked)
    remove_edge_button.on_click(remove_edge_button_clicked)

    # Display the widgets
    display(widgets.VBox([node_input, add_node_button, remove_node_button, edge_input, add_edge_button, remove_edge_button, highlight_checkbox, graph_output, centrality_output]))


In [ ]:
# import networkx as nx
# from interface import Interface_BC
# from Brandes_CB import Brandes_CB_algoritm

# G = nx.karate_club_graph()
# G = nx.davis_southern_women_graph()
# G = nx.les_miserables_graph()
# G = nx.florentine_families_graph()
G = nx.bull_graph()
k1=0.1

Interface_BC1(G, Brandes_CB_algoritm, k1, size_i=5)


In [ ]:
import urllib.request
import io
import zipfile

import matplotlib.pyplot as plt
import networkx as nx

url = "http://www-personal.umich.edu/~mejn/netdata/football.zip"

sock = urllib.request.urlopen(url)  # open URL
s = io.BytesIO(sock.read())  # read into BytesIO "file"
sock.close()

zf = zipfile.ZipFile(s)  # zipfile object
txt = zf.read("football.txt").decode()  # read info file
gml = zf.read("football.gml").decode()  # read gml data
# throw away bogus first line with # from mejn files
gml = gml.split("\n")[1:]
G = nx.parse_gml(gml)  # parse gml data

Interface_BC1(G, Brandes_CB_algoritm, k1=1.7, size_i = 15)